# Hugging Face Summarization: Models that *actually* work (+ correct `pipeline` task names)

**Generated:** 2025-09-15 02:52:46

This notebook gives you:
- A short, practical model list that works well for summarization
- The **correct `pipeline` task name** for each (e.g., `"summarization"` vs `"text2text-generation"`)
- Minimal code cells you can run directly
- Tips for long documents (chunking / map-reduce)


## How to run this notebook

1. Make sure you have recent versions of `transformers` (and optionally `accelerate`, `sentencepiece` for T5/PEGASUS/mT5) installed.  
2. The first time you run a model, it will download weights from the Hugging Face Hub (internet required).  
3. GPU is optional but recommended for speed; otherwise CPU will work but slower.

> **Install (optional, run once):**
```bash
pip install -U transformers accelerate sentencepiece
```


## Models by use-case (with correct `pipeline` task)

### Short/medium articles (general purpose)
- **`facebook/bart-large-cnn`** — task: **`"summarization"`**  
Reliable “news-style” multi-sentence summaries.
- **`sshleifer/distilbart-cnn-12-6`** — task: **`"summarization"`**  
Faster, smaller BART; great quality/speed tradeoff.
- **`google/pegasus-cnn_dailymail`** — task: **`"summarization"`**  
Strong abstractive summaries in similar domain.

### One-sentence / ultra-concise “headline” style
- **`google/pegasus-xsum`** — task: **`"summarization"`**  
Trained for **single-sentence** extreme summaries.

### Prompt-steerable (add your own style/instructions)
- **`google/flan-t5-base`** / **`google/flan-t5-large`** — task: **`"text2text-generation"`**  
Use a custom prompt like: *“Summarize in one neutral sentence for an executive.”*  
  (T5 also works with `"summarization"`, but `"text2text-generation"` gives explicit prompt control.)

### Long documents (reports, papers, chapters)
- **`google/bigbird-pegasus-large-arxiv`** — task: **`"summarization"`**  
Handles longer inputs better; good for scientific/longer texts.
- **`pszemraj/led-large-book-summary`** — task: **`"summarization"`**  
LED variant tuned for long-form/chapters/book-like content.

### Dialogues / meetings / chats
- **`google/pegasus-samsum`** — task: **`"summarization"`**  
Built for conversation transcripts (SAMSum dataset).

### Multilingual summarization
- **`csebuetnlp/mT5_multilingual_XLSum`** — task: **`"summarization"`** *or* **`"text2text-generation"`**  
Works across many languages; use `"text2text-generation"` if you want instruction prompts.


In [ ]:
# Optional: install/upgrade dependencies. Uncomment to run here.
# %pip install -U transformers accelerate sentencepiece


In [ ]:
# ▶️ Quick sample texts for testing
long_text = (
    "Open-source tools have accelerated AI adoption across industries. "
    "Teams use models for summarization, search, and code assistance. "
    "However, deploying models at scale requires attention to latency, cost, and governance. "
    "In this article, we explore tradeoffs between speed and quality, and offer simple recipes."
)

dialogue_text = (
    "Alice: Hey, did you finish the quarterly report?\n"
    "Bob: Almost. I still have to add the revenue projections.\n"
    "Alice: Can you share a brief summary for the leadership update?\n"
    "Bob: Sure, I'll include a one-paragraph summary and send it by 5 PM."
)

multilingual_text = "El proyecto avanza según lo previsto, pero necesitamos más pruebas de integración antes del lanzamiento."


### Example 1 — BART (general purpose) · task = `"summarization"`

In [ ]:
from transformers import pipeline

# News-style multi-sentence summaries
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

result = summarizer(long_text, max_length=180, min_length=60, do_sample=False)
print(result[0]["summary_text"])

### Example 2 — DistilBART (faster) · task = `"summarization"`

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
print(summarizer(long_text, max_length=150, min_length=50, do_sample=False)[0]["summary_text"])

### Example 3 — PEGASUS CNN/DailyMail · task = `"summarization"`

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="google/pegasus-cnn_dailymail")
print(summarizer(long_text, max_length=180, min_length=60, do_sample=False)[0]["summary_text"])

### Example 4 — PEGASUS XSum (single-sentence) · task = `"summarization"`

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="google/pegasus-xsum")
print(summarizer(long_text, max_length=60, min_length=8, do_sample=False)[0]["summary_text"])

### Example 5 — FLAN‑T5 with prompt control · task = `"text2text-generation"`

In [ ]:
from transformers import pipeline

# Use a custom prompt to steer style/tone/length
summarizer = pipeline("text2text-generation", model="google/flan-t5-large")

PROMPT = "Summarize in one neutral sentence for an executive:"
out = summarizer(f"{PROMPT}\n\n{long_text}", max_new_tokens=80)
print(out[0]["generated_text"].strip())

### Example 6 — BigBird‑Pegasus for longer inputs · task = `"summarization"`

In [ ]:
from transformers import pipeline
from textwrap import wrap

# ⚠️ Simple word-based chunking utility (approximate). For production,
# consider tokenization-aware chunking to respect model limits.
def chunk_text(text, words_per_chunk=250, overlap=30):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+words_per_chunk]
        chunks.append(" ".join(chunk))
        i += (words_per_chunk - overlap)
    return chunks

# Example long-ish input (reuse long_text * N for demonstration)
very_long_text = " ".join([long_text] * 20)

summarizer = pipeline("summarization", model="google/bigbird-pegasus-large-arxiv")

# Map step: summarize each chunk
pieces = chunk_text(very_long_text, words_per_chunk=250, overlap=30)
piece_summaries = [summarizer(p, max_length=160, min_length=60, do_sample=False)[0]["summary_text"] for p in pieces]

# Reduce step: summarize the summaries
joined = " ".join(piece_summaries)
final_summary = summarizer(joined, max_length=200, min_length=80, do_sample=False)[0]["summary_text"]
print(final_summary)

### Example 7 — LED (book-like chapters) · task = `"summarization"`

In [ ]:
from transformers import pipeline

# LED model tuned for long-form content
summarizer = pipeline("summarization", model="pszemraj/led-large-book-summary")

# For very long book chapters, chunk first (as above), then map-reduce
print(summarizer(long_text, max_length=200, min_length=80, do_sample=False)[0]["summary_text"])

### Example 8 — SAMSum (dialogues/meetings) · task = `"summarization"`

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="google/pegasus-samsum")
print(summarizer(dialogue_text, max_length=120, min_length=40, do_sample=False)[0]["summary_text"])

### Example 9 — mT5 Multilingual · task = `"summarization"` *or* `"text2text-generation"`

In [ ]:
from transformers import pipeline

# Use 'text2text-generation' if you want to add explicit prompts in various languages
summarizer = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum")
print(summarizer(multilingual_text, max_length=80, min_length=20, do_sample=False)[0]["summary_text"])

## Quick tips

- **Task names to remember**  
  - Most ready-to-use summarizers → **`"summarization"`**  
  - T5/FLAN‑T5 when you want explicit prompts → **`"text2text-generation"`**
- **Speed vs. quality**  
  Start with `sshleifer/distilbart-cnn-12-6` for speed, then try `facebook/bart-large-cnn` for a quality bump.
- **Long inputs**  
  Use chunking + map-reduce: summarize chunks → join → summarize the summaries. Overlap chunks to reduce boundary loss.
- **Deterministic output**  
  Set `do_sample=False` (BART/PEGASUS) or keep temperatures low for T5-style models.
- **GPU**  
  Add `device=0` to the `pipeline(...)` call (if CUDA is available) for significant speedups.


### Bonus — Generic map-reduce summarization utility

In [ ]:
from transformers import pipeline

def map_reduce_summarize(text, model_name="facebook/bart-large-cnn",
                         words_per_chunk=250, overlap=30,
                         map_kwargs=None, reduce_kwargs=None,
                         task="summarization"):
    """Generic chunk->summarize->join->summarize pipeline.
    - task: 'summarization' or 'text2text-generation' depending on your model
    - map_kwargs/reduce_kwargs: dicts forwarded to pipeline calls
    """
    if map_kwargs is None:
        map_kwargs = dict(max_length=160, min_length=60, do_sample=False)
    if reduce_kwargs is None:
        reduce_kwargs = dict(max_length=200, min_length=80, do_sample=False)

    summ = pipeline(task, model=model_name)

    # Chunk
    def chunk_text(text, words_per_chunk=250, overlap=30):
        words = text.split()
        chunks = []
        i = 0
        while i < len(words):
            chunk = words[i:i+words_per_chunk]
            chunks.append(" ".join(chunk))
            i += (words_per_chunk - overlap)
        return chunks

    pieces = chunk_text(text, words_per_chunk=words_per_chunk, overlap=overlap)

    # Map
    piece_summaries = [summ(p, **map_kwargs)[0]["summary_text"] for p in pieces]

    # Reduce
    joined = " ".join(piece_summaries)
    final = summ(joined, **reduce_kwargs)[0]["summary_text"]
    return final

# Demo (small text here, but this shines on very long inputs)
print(map_reduce_summarize(long_text))